# Análisis de Departamentos y municipios de Colombia

## Integrantes - Grupo 15

- Sara Sofia Cárdenas Rodríguez - 202214907
- Daniel Felipe Diaz Moreno - 202210773
- Juan Sebastián Urrea López - 201914710


## Los datos

Se trabaja con un conjunto de datos que contiene las regiones, departamentos y municipios de Colombia, asi como sus correspondientes identificadores


## 1. Carga y Manipulación básica de los Datos

In [92]:
# Importaciones de bibliotecas
import pandas as pd

In [93]:
regiones_df = pd.read_csv("Regiones.csv", delimiter=",")


print(f"Las dimensiones de Regiones.csv son: {regiones_df.shape}")


Las dimensiones de Regiones.csv son: (1123, 5)


In [94]:
regiones_df.sample(10)

,REGION,CÓDIGO DANE DEL DEPARTAMENTO,DEPARTAMENTO,CÓDIGO DANE DEL MUNICIPIO,MUNICIPIO
369,Región Caribe,20,Cesar,20.550,Pelaya
282,Región Eje Cafetero - Antioquia,17,Caldas,17.088,Belalcázar
928,Región Caribe,70,Sucre,70.702,San Juan de Betulia
497,Región Pacífico,27,Chocó,27.001,Quibdó
149,Región Caribe,13,Bolívar,13.433,Mahates
13,Región Eje Cafetero - Antioquia,5,Antioquia,5.051,Arboletes
1055,Región Centro Sur,86,Putumayo,86.568,Puerto Asís
600,Región Llano,50,Meta,50.313,Granada
116,Región Eje Cafetero - Antioquia,5,Antioquia,5.885,Yalí
176,Región Centro Oriente,15,Boyacá,15.097,Boavita


In [95]:
regiones_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123 entries, 0 to 1122
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   REGION                        1123 non-null   object 
 1   CÓDIGO DANE DEL DEPARTAMENTO  1123 non-null   int64  
 2   DEPARTAMENTO                  1123 non-null   object 
 3   CÓDIGO DANE DEL MUNICIPIO     1123 non-null   float64
 4   MUNICIPIO                     1123 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 44.0+ KB


Vemos que los tipos de atributos son consistentes con lo esperados.

In [96]:
atributosDeTexto = regiones_df.select_dtypes(include=['object']).columns
regiones_df[atributosDeTexto].describe()


,REGION,DEPARTAMENTO,MUNICIPIO
count,1123,1123,1123
unique,6,33,1035
top,Región Centro Oriente,Antioquia,La Unión
freq,367,125,4


Vemos que "La Unión" aparece 4 veces, posteriormente se evaluará si es un error de unicidad o si se debe a municipios homonimos

## 2. Entendimiento de los datos

## 2.1. Análisis de calidad de datos





## 2.1.1. Análisis de completitud
A continuación, se presentan los porcentajes de completitud e incompletitud para cada uno de los atributos.

In [97]:
porcentajeIncompletitudAtributo =  (100 * regiones_df.isnull().sum() / regiones_df.shape[0]).sort_values(ascending=False)
pd.DataFrame({
    "Atributo": porcentajeIncompletitudAtributo.index,
    "Completitud (%)": 100 - porcentajeIncompletitudAtributo.values,
    "Incompletitud (%)": porcentajeIncompletitudAtributo.values
})

,Atributo,Completitud (%),Incompletitud (%)
0,REGION,100.0,0.0
1,CÓDIGO DANE DEL DEPARTAMENTO,100.0,0.0
2,DEPARTAMENTO,100.0,0.0
3,CÓDIGO DANE DEL MUNICIPIO,100.0,0.0
4,MUNICIPIO,100.0,0.0


Todos los datos estan presentes, no hay problemas de completitud

## 2.1.2. Análisis de unicidad
A continuación, se buscará determinar cuántas filas duplicadas hay en el dataset

In [98]:
registrosDuplicados = regiones_df.duplicated()
totalDuplicados = registrosDuplicados.sum()
totalOpiniones = regiones_df.shape[0]
porcentajeDuplicados = 100.0 * totalDuplicados / totalOpiniones
print(f"Se cuenta con {totalDuplicados} registros duplicados ({porcentajeDuplicados:.2f}%)")



Se cuenta con 0 registros duplicados (0.00%)


In [99]:
registrosDuplicados = regiones_df[["MUNICIPIO","DEPARTAMENTO"]].duplicated()
totalDuplicados = registrosDuplicados.sum()
totalOpiniones = regiones_df[["MUNICIPIO","DEPARTAMENTO"]].shape[0]
porcentajeDuplicados = 100.0 * totalDuplicados / totalOpiniones
print(f"Se cuenta con {totalDuplicados} registros duplicados ({porcentajeDuplicados:.2f}%)")


Se cuenta con 0 registros duplicados (0.00%)


No hay registros duplicados, no hay problemas de unicidad.
Tambien se puede ver que los municipios repetidos, son municipios homonimos, no se trata de municipios duplicados

## 2.1.3. Análisis de validez
No se encontraron errores de validez


## 2.1.4. Análisis de consistencia

No se encontraron errores de consistencia

# 3 Preparación de los datos
Si bien no existen problemas de calidad en los datos, el formato difiere con las fuentes principales. Por ejemplo, el archivo de defunciones se usan los codigos de municipio sin tener en cuenta el departamento, mientras que en los datos analizados en este notebook cada codigo del municipio inicia con el codigo del departamento. Para facilitiar la interoperabilidad, es necesario hacer que los formatos sean los mismos.

In [100]:

regiones_df["CÓDIGO DANE DEL MUNICIPIO"] = regiones_df["CÓDIGO DANE DEL MUNICIPIO"].apply(lambda x: str(x).split('.')[1].ljust(3, '0') if '.' in str(x) else str(x).zfill(3))
regiones_df["CÓDIGO DANE DEL MUNICIPIO"]


0       001
1       002
2       004
3       021
4       030
       ... 
1118    444
1119    051
1120    109
1121    189
1122    560
Name: CÓDIGO DANE DEL MUNICIPIO, Length: 1123, dtype: object

Finalmente, se exporta el dataframe con estos cambios

In [101]:
regiones_df.to_csv("RegionesFormato.csv", index=False)

# 4. Resumen de los cambios realizados
Al haber analizado los datos, se encuentra que no se deben realizar modificaciones adicionales.